In [1]:
import numpy as np 
import time
import os

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d

# DeepDense VAE encoder + classifier

In [2]:
X_train = np.load("data/encoded_data_train_DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.npy")
y_train = np.load("data/labels_train.npy")
X_test = np.load("data/encoded_data_test_DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.npy")
y_test = np.load("data/labels_test.npy")

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [4]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [31]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=128, shuffle=True)

# M.1

In [45]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=125),
            nn.ELU(inplace=True),
            nn.Linear(in_features=125, out_features=4)
)

In [46]:
optimizer= torch.optim.Adam(DNN1.parameters(), lr=1e-2)

In [47]:
m1 = Trainer(
        model=DNN1,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.I_lr-{1e-2,1e-3}_bs-128",
        verbose=True
        )

cuda:0


In [48]:
lh = m1(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6062, validation_loss:0.5582, val_accuracy:0.8312
Epoch [2/100], average_loss:0.5459, validation_loss:0.5651, val_accuracy:0.7532
Epoch [3/100], average_loss:0.5293, validation_loss:0.5165, val_accuracy:0.7792
Epoch [4/100], average_loss:0.4988, validation_loss:0.4945, val_accuracy:0.8312
Epoch [5/100], average_loss:0.4801, validation_loss:0.4879, val_accuracy:0.8961
Epoch [6/100], average_loss:0.4702, validation_loss:0.4738, val_accuracy:0.7532
Epoch [7/100], average_loss:0.4627, validation_loss:0.4772, val_accuracy:0.8312
Epoch [8/100], average_loss:0.4586, validation_loss:0.4589, val_accuracy:0.8052
Epoch [9/100], average_loss:0.4569, validation_loss:0.4642, val_accuracy:0.7922
Epoch [10/100], average_loss:0.4568, validation_loss:0.4549, val_accuracy:0.7403
Epoch [11/100], average_loss:0.4303, validation_loss:0.4397, val_accuracy:0.8701
Epoch [12/100], average_loss:0.4273, validation_loss:0.4388, val_accuracy:0.7532
Epoch [13/100], average_loss:0.4262, 

In [36]:
m1.tb.close()

In [49]:
m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)


In [50]:
x_pred

tensor([1, 1, 1,  ..., 3, 0, 0])

In [51]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8016459924684606

In [52]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.842315965583174

In [53]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2698, 1392,   21,  124],
       [ 454, 9813,   37,  141],
       [  28,   49,  381,   21],
       [ 195,  159,   18, 1205]], dtype=int64)

In [54]:
#torch.save(m1.model, "models_and_losses/DNN_latent_space_m.I_lr-{1e-2,1e-3}_bs-128_epoch-100.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ELU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# M.2

In [55]:
DNN2 = nn.Sequential(
            nn.Linear(in_features=15, out_features=4),
)

In [56]:
optimizer= torch.optim.Adam(DNN2.parameters(), lr=1e-2)

In [57]:
m2 = Trainer(
        model=DNN2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.II_lr-1e-2_bs-128",
        verbose=True
        )

cuda:0


In [58]:
lh = m2(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.8076, validation_loss:0.6691, val_accuracy:0.7143
Epoch [2/100], average_loss:0.6611, validation_loss:0.6500, val_accuracy:0.7403
Epoch [3/100], average_loss:0.6511, validation_loss:0.6477, val_accuracy:0.7143
Epoch [4/100], average_loss:0.6483, validation_loss:0.6420, val_accuracy:0.6883
Epoch [5/100], average_loss:0.6461, validation_loss:0.6427, val_accuracy:0.6104
Epoch [6/100], average_loss:0.6455, validation_loss:0.6428, val_accuracy:0.8052
Epoch [7/100], average_loss:0.6440, validation_loss:0.6366, val_accuracy:0.7662
Epoch [8/100], average_loss:0.6439, validation_loss:0.6391, val_accuracy:0.7922
Epoch [9/100], average_loss:0.6435, validation_loss:0.6374, val_accuracy:0.7532
Epoch [10/100], average_loss:0.6434, validation_loss:0.6365, val_accuracy:0.7792
Epoch [11/100], average_loss:0.6430, validation_loss:0.6371, val_accuracy:0.7403
Epoch [12/100], average_loss:0.6428, validation_loss:0.6386, val_accuracy:0.7532
Epoch [13/100], average_loss:0.6430, 

In [59]:
m2.tb.close()

In [60]:
m2.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float().to(m2.device)).cpu().detach(), axis=1)


In [61]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [62]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.7008171233424978

In [63]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.7609942638623327

In [64]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[2055, 2102,   15,   63],
       [ 703, 9456,   47,  239],
       [  47,   79,  342,   11],
       [ 250,  430,   14,  883]], dtype=int64)

# M.3

In [67]:
DNN3 = nn.Sequential(
            nn.Linear(in_features=15, out_features=125),
            nn.Sigmoid(),
            nn.Linear(in_features=125, out_features=4),
)

In [68]:
optimizer= torch.optim.Adam(DNN3.parameters(), lr=1e-2)

In [69]:
m3 = Trainer(
        model=DNN3,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.III_lr-1e-2_bs-128",
        verbose=True
        )

cuda:0


In [70]:
lh = m3(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6496, validation_loss:0.5666, val_accuracy:0.7532
Epoch [2/100], average_loss:0.5141, validation_loss:0.4692, val_accuracy:0.8701
Epoch [3/100], average_loss:0.4590, validation_loss:0.4450, val_accuracy:0.8442
Epoch [4/100], average_loss:0.4354, validation_loss:0.4358, val_accuracy:0.8052
Epoch [5/100], average_loss:0.4229, validation_loss:0.4323, val_accuracy:0.7792
Epoch [6/100], average_loss:0.4152, validation_loss:0.4302, val_accuracy:0.8961
Epoch [7/100], average_loss:0.4068, validation_loss:0.4213, val_accuracy:0.8182
Epoch [8/100], average_loss:0.4019, validation_loss:0.4154, val_accuracy:0.9221
Epoch [9/100], average_loss:0.3947, validation_loss:0.4156, val_accuracy:0.8052
Epoch [10/100], average_loss:0.3895, validation_loss:0.4016, val_accuracy:0.8701
Epoch [11/100], average_loss:0.3867, validation_loss:0.3948, val_accuracy:0.8571
Epoch [12/100], average_loss:0.3807, validation_loss:0.4061, val_accuracy:0.8961
Epoch [13/100], average_loss:0.3775, 

In [71]:
m3.tb.close()

In [72]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().to(m3.device)).cpu().detach(), axis=1)


In [73]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [74]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8089129250103856

In [75]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8469168260038241

In [76]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3197,  864,   14,  160],
       [ 866, 9401,   47,  131],
       [  37,   35,  379,   28],
       [ 198,  169,   13, 1197]], dtype=int64)

## lr=0.001

In [77]:
DNN3 = nn.Sequential(
            nn.Linear(in_features=15, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [78]:
optimizer= torch.optim.Adam(DNN3.parameters(), lr=1e-2)

In [79]:
m3 = Trainer(
        model=DNN3,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.III_lr-{1e-2,1e-3}_bs-128",
        verbose=True
        )

cuda:0


In [80]:
lh = m3(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6570, validation_loss:0.5778, val_accuracy:0.7403
Epoch [2/100], average_loss:0.5137, validation_loss:0.4833, val_accuracy:0.8442
Epoch [3/100], average_loss:0.4579, validation_loss:0.4431, val_accuracy:0.8442
Epoch [4/100], average_loss:0.4352, validation_loss:0.4391, val_accuracy:0.8182
Epoch [5/100], average_loss:0.4221, validation_loss:0.4272, val_accuracy:0.7792
Epoch [6/100], average_loss:0.4141, validation_loss:0.4139, val_accuracy:0.8312
Epoch [7/100], average_loss:0.4032, validation_loss:0.4144, val_accuracy:0.8571
Epoch [8/100], average_loss:0.3956, validation_loss:0.4083, val_accuracy:0.8831
Epoch [9/100], average_loss:0.3909, validation_loss:0.4016, val_accuracy:0.8312
Epoch [10/100], average_loss:0.3845, validation_loss:0.3977, val_accuracy:0.8442
Epoch [11/100], average_loss:0.3794, validation_loss:0.3954, val_accuracy:0.8052
Epoch [12/100], average_loss:0.3751, validation_loss:0.3955, val_accuracy:0.8442
Epoch [13/100], average_loss:0.3709, 

In [81]:
m3.tb.close()

In [82]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().to(m3.device)).cpu().detach(), axis=1)


In [83]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [84]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8104041592404143

In [85]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8503226577437859

In [86]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3021, 1060,   26,  128],
       [ 643, 9615,   51,  136],
       [  23,   46,  395,   15],
       [ 214,  140,   23, 1200]], dtype=int64)

## scheduler and bs = 256

In [92]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=256, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=256, shuffle=True)

In [93]:
DNN3 = nn.Sequential(
            nn.Linear(in_features=15, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [94]:
optimizer= torch.optim.Adam(DNN3.parameters(), lr=1e-2)

In [95]:
m3 = Trainer(
        model=DNN3,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.III_lr-{1e-2,1e-3}_bs-256",
        verbose=True
        )

cuda:0


In [96]:
lh = m3(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6863, validation_loss:0.6075, val_accuracy:0.8182
Epoch [2/100], average_loss:0.5881, validation_loss:0.5502, val_accuracy:0.7403
Epoch [3/100], average_loss:0.5102, validation_loss:0.4767, val_accuracy:0.8571
Epoch [4/100], average_loss:0.4653, validation_loss:0.4579, val_accuracy:0.8701
Epoch [5/100], average_loss:0.4437, validation_loss:0.4435, val_accuracy:0.8312
Epoch [6/100], average_loss:0.4339, validation_loss:0.4407, val_accuracy:0.8182
Epoch [7/100], average_loss:0.4250, validation_loss:0.4409, val_accuracy:0.8182
Epoch [8/100], average_loss:0.4191, validation_loss:0.4236, val_accuracy:0.8442
Epoch [9/100], average_loss:0.4129, validation_loss:0.4120, val_accuracy:0.8961
Epoch [10/100], average_loss:0.4048, validation_loss:0.4309, val_accuracy:0.7662
Epoch [11/100], average_loss:0.4003, validation_loss:0.4080, val_accuracy:0.8831
Epoch [12/100], average_loss:0.3931, validation_loss:0.4106, val_accuracy:0.9091
Epoch [13/100], average_loss:0.3911, 

In [97]:
m3.tb.close()

In [98]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().to(m3.device)).cpu().detach(), axis=1)


In [99]:
x_pred

tensor([1, 0, 1,  ..., 0, 1, 0])

In [100]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8132117105214246

In [101]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8519359464627151

In [102]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3002, 1093,   29,  111],
       [ 608, 9677,   47,  113],
       [  19,   48,  393,   19],
       [ 227,  151,   13, 1186]], dtype=int64)

In [104]:
#torch.save(m3.model, "models_and_losses/DNN_latent_space_m.III_lr-{1e-2,1e-3}_bs-256_epoch-100.pt")

# M.4

In [116]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=128, shuffle=True)

In [117]:
DNN4 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [118]:
optimizer= torch.optim.Adam(DNN4.parameters(), lr=1e-2)

In [119]:
m4 = Trainer(
        model=DNN4,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.IV_lr-{e-2,1e-3}_epochs-128",
        verbose=True
        )

cuda:0


In [120]:
lh = m4(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6335, validation_loss:0.5137, val_accuracy:0.8052
Epoch [2/100], average_loss:0.4703, validation_loss:0.4434, val_accuracy:0.8052
Epoch [3/100], average_loss:0.4298, validation_loss:0.4209, val_accuracy:0.8442
Epoch [4/100], average_loss:0.4085, validation_loss:0.4116, val_accuracy:0.8182
Epoch [5/100], average_loss:0.3995, validation_loss:0.3919, val_accuracy:0.9091
Epoch [6/100], average_loss:0.3875, validation_loss:0.4124, val_accuracy:0.8052
Epoch [7/100], average_loss:0.3783, validation_loss:0.3811, val_accuracy:0.8442
Epoch [8/100], average_loss:0.3693, validation_loss:0.3838, val_accuracy:0.9091
Epoch [9/100], average_loss:0.3625, validation_loss:0.3811, val_accuracy:0.8312
Epoch [10/100], average_loss:0.3557, validation_loss:0.3750, val_accuracy:0.8182
Epoch [11/100], average_loss:0.3512, validation_loss:0.3826, val_accuracy:0.8961
Epoch [12/100], average_loss:0.3441, validation_loss:0.3624, val_accuracy:0.8831
Epoch [13/100], average_loss:0.3402, 

In [121]:
m4.tb.close()

In [122]:
m4.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float().to(m4.device)).cpu().detach(), axis=1)


In [123]:
x_pred

tensor([1, 0, 1,  ..., 3, 1, 0])

In [124]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8247516540094679

In [125]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8638264818355641

In [126]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3078,  989,   34,  134],
       [ 543, 9763,   33,  106],
       [  20,   49,  396,   14],
       [ 189,  148,   20, 1220]], dtype=int64)

In [127]:
#torch.save(m4.model, "models_and_losses/DNN_latent_space_m.IV_lr-{e-2,1e-3}_epochs-128_epoch-100.pt")

## Bs = 256

In [128]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=256, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=256, shuffle=True)

In [129]:
DNN4 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [130]:
optimizer= torch.optim.Adam(DNN4.parameters(), lr=1e-2)

In [131]:
m4 = Trainer(
        model=DNN4,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.IV_lr-{1e-2,1e-3}_epochs-256",
        verbose=True
        )

cuda:0


In [132]:
lh = m4(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.7086, validation_loss:0.5749, val_accuracy:0.8182
Epoch [2/100], average_loss:0.5248, validation_loss:0.4786, val_accuracy:0.8442
Epoch [3/100], average_loss:0.4574, validation_loss:0.4331, val_accuracy:0.8831
Epoch [4/100], average_loss:0.4263, validation_loss:0.4247, val_accuracy:0.7922
Epoch [5/100], average_loss:0.4141, validation_loss:0.4140, val_accuracy:0.8831
Epoch [6/100], average_loss:0.4006, validation_loss:0.4383, val_accuracy:0.8442
Epoch [7/100], average_loss:0.3928, validation_loss:0.3871, val_accuracy:0.9221
Epoch [8/100], average_loss:0.3801, validation_loss:0.3881, val_accuracy:0.8831
Epoch [9/100], average_loss:0.3734, validation_loss:0.3767, val_accuracy:0.8312
Epoch [10/100], average_loss:0.3661, validation_loss:0.3722, val_accuracy:0.8442
Epoch [11/100], average_loss:0.3589, validation_loss:0.3697, val_accuracy:0.8182
Epoch [12/100], average_loss:0.3507, validation_loss:0.3639, val_accuracy:0.9221
Epoch [13/100], average_loss:0.3474, 

In [133]:
m4.tb.close()

In [134]:
m4.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float().to(m4.device)).cpu().detach(), axis=1)


In [135]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [136]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8277852135595489

In [137]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.863527724665392

In [138]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3198,  895,   21,  121],
       [ 643, 9663,   24,  115],
       [  30,   46,  383,   20],
       [ 217,  143,    9, 1208]], dtype=int64)

In [139]:
#torch.save(m4.model, "models_and_losses/DNN_latent_space_m.IV_lr-{1e-2,1e-3}_epochs-256_epoch-100.pt")

## Bs =512

In [140]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [141]:
DNN4 = nn.Sequential(
            nn.Linear(in_features=15, out_features=64),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4),
)

In [142]:
optimizer= torch.optim.Adam(DNN4.parameters(), lr=1e-2)

In [143]:
m4 = Trainer(
        model=DNN4,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.IV_lr-{1e-2,1e-3}_epochs-512",
        verbose=True
        )

cuda:0


In [144]:
lh = m4(epochs=100, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.7981, validation_loss:0.6241, val_accuracy:0.6753
Epoch [2/100], average_loss:0.5909, validation_loss:0.5516, val_accuracy:0.7922
Epoch [3/100], average_loss:0.5296, validation_loss:0.4997, val_accuracy:0.8571
Epoch [4/100], average_loss:0.4792, validation_loss:0.4567, val_accuracy:0.7662
Epoch [5/100], average_loss:0.4441, validation_loss:0.4457, val_accuracy:0.8052
Epoch [6/100], average_loss:0.4327, validation_loss:0.4442, val_accuracy:0.7792
Epoch [7/100], average_loss:0.4206, validation_loss:0.4179, val_accuracy:0.8961
Epoch [8/100], average_loss:0.4126, validation_loss:0.4270, val_accuracy:0.6883
Epoch [9/100], average_loss:0.4049, validation_loss:0.4157, val_accuracy:0.8052
Epoch [10/100], average_loss:0.3973, validation_loss:0.4104, val_accuracy:0.8442
Epoch [11/100], average_loss:0.3901, validation_loss:0.3978, val_accuracy:0.8701
Epoch [12/100], average_loss:0.3851, validation_loss:0.3926, val_accuracy:0.8571
Epoch [13/100], average_loss:0.3795, 

In [145]:
m4.tb.close()

In [146]:
m4.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float().to(m4.device)).cpu().detach(), axis=1)


In [147]:
x_pred

tensor([1, 0, 1,  ..., 3, 1, 0])

In [148]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8301873442078451

In [149]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8630497131931166

In [150]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3111,  961,   25,  138],
       [ 622, 9689,   25,  109],
       [  20,   46,  392,   21],
       [ 177,  135,   13, 1252]], dtype=int64)

In [151]:
#torch.save(m4.model, "models_and_losses/DNN_latent_space_m.IV_lr-{1e-2,1e-3}_epochs-512_epoch-100.pt")